# 🎯 JetX Progressive Training - Google Colab'da Model Eğitimi

**Bu notebook Progressive Training stratejisi ile JetX modelini Google Colab'da eğitir.**

## 📊 Sorun:

Mevcut model **1.5 altı değerleri tahmin edemiyor** - Hep 1.5 üstü tahmin ediyor!

## 🎯 Çözüm: 3 Aşamalı Progressive Training

### AŞAMA 1: Foundation Training (100 epoch) ⏱️ ~30 dakika

- Model hem **değer tahmin etmeyi** HEM DE **1.5 eşiğini** birlikte öğrenir
- Learning Rate: 0.0001
- Batch Size: 64
- Loss Weight: Regression 60%, Classification 10%, Threshold 30%
- **Threshold baştan aktif!** (Eski yöntem: sadece regression)
- Monitor: val_threshold_accuracy | Patience: 10

### AŞAMA 2: Threshold Fine-Tuning (80 epoch) ⏱️ ~25 dakika

- **1.5 altı/üstü ayrımını** keskinleştirir (yumuşak strateji)
- Learning Rate: 0.00005
- Batch Size: 32
- Class Weight: **5x** (yumuşak - agresif değil!)
- Loss Weight: Regression 40%, Threshold 60%
- Monitor: val_threshold_accuracy | Patience: 10

### AŞAMA 3: Full Model Fine-Tuning (80 epoch) ⏱️ ~40 dakika

- **Tüm output'ları** birlikte optimize eder (dengeli final push)
- Learning Rate: 0.00003
- Batch Size: 16
- Class Weight: **7x** (dengeli!)
- Loss Weight: Regression 30%, Classification 15%, Threshold 55%
- Monitor: val_threshold_accuracy | Patience: 10

## 🎯 Hedefler:

- 🔴 **1.5 Altı Doğruluk**: %70-80%+
- 🟢 **1.5 Üstü Doğruluk**: %75-85%+
- 💰 **Para Kaybı Riski**: %20 altı
- 📊 **MAE**: < 2.0

## ⏱️ Toplam Süre: ~1.5 saat (GPU ile)

---

## 📦 Adım 1: Hazırlık (2-3 dakika)

Aşağıdaki hücreyi çalıştırın. Kütüphaneler yüklenecek ve proje hazırlanacak.

In [ ]:
#@title ▶️ Hazırlık - Buraya tıklayın { display-mode: "form" }

print("📦 Kütüphaneler yükleniyor...")
!pip install -q tensorflow scikit-learn pandas numpy scipy joblib matplotlib seaborn tqdm

print("\n📥 Proje GitHub'dan indiriliyor...")
import os
if os.path.exists('jetxpredictor'):
    !rm -rf jetxpredictor
!git clone https://github.com/onndd/jetxpredictor.git
%cd jetxpredictor

print("\n✅ Hazırlık tamamlandı!")
print("\n➡️ Şimdi bir sonraki hücreye geçin (aşağıdaki ▶️ butonuna tıklayın)")

## 🔥 Adım 2: Progressive Training (1.5 saat)

Aşağıdaki hücreyi çalıştırın. 3 aşamalı model eğitimi başlayacak.

**⚠️ ÖNEMLİ:**

- Bu işlem 1.5 saat sürecek
- Tarayıcınızı kapatmayın!
- Her 5 epoch'ta detaylı metrik açıklamaları göreceksiniz
- 3 aşama sırayla otomatik çalışacak

In [ ]:
#@title ▶️ Progressive Training Başlat - Buraya tıklayın { display-mode: "form" }

print("🎯 PROGRESSIVE TRAINING Başlatılıyor...")
print("⏱️ Tahmini süre: 1.5 saat (GPU ile)")
print("📊 İlerlemeyi aşağıda görebilirsiniz\n")
print("="*70)

!python notebooks/jetx_PROGRESSIVE_TRAINING.py

print("\n" + "="*70)
print("✅ Progressive Training tamamlandı!")
print("\n➡️ Şimdi bir sonraki hücreye geçin (aşağıdaki ▶️ butonuna tıklayın)")

## 💾 Adım 3: Modeli İndir

Aşağıdaki hücreyi çalıştırın. Eğitilmiş model dosyaları bilgisayarınıza indirilecek.

**İndirilen dosyalar:**
- `jetx_progressive_final.h5` - Ana model dosyası
- `scaler_progressive.pkl` - Normalizer
- `progressive_model_info.json` - Model bilgileri

In [ ]:
#@title ▶️ Dosyaları İndir - Buraya tıklayın { display-mode: "form" }

from google.colab import files
import os

print("💾 Dosyalar indiriliyor...\n")

model_files = [
    'jetx_progressive_final.h5',
    'scaler_progressive.pkl',
    'progressive_model_info.json'
]

for file in model_files:
    if os.path.exists(file):
        print(f"📥 {file} indiriliyor...")
        files.download(file)
    else:
        print(f"⚠️ {file} bulunamadı!")

print("\n✅ Tüm dosyalar bilgisayarınıza indirildi!")
print("\n📁 İndirilen dosyalar:")
print("   • jetx_progressive_final.h5 (Ana model)")
print("   • scaler_progressive.pkl (Normalizer)")
print("   • progressive_model_info.json (Model bilgileri)")
print("\n➡️ Bu dosyaları lokal projenizin 'models/' klasörüne kopyalayın:")
print("   1. jetx_progressive_final.h5 → models/jetx_model.h5")
print("   2. scaler_progressive.pkl → models/scaler.pkl")
print("\n🎉 Tamamlandı! Artık yeni modeli kullanabilirsiniz.")

---

## 📊 Metrik Açıklamaları

Eğitim sırasında her 5 epoch'ta göreceğiniz metriklerin anlamları:

### 🔴 1.5 ALTI DOĞRULUĞU

**Ne anlama geliyor?**
- Model, **gerçekten 1.5 altında olan** değerleri ne kadar doğru tahmin ediyor?
- Örnek: 100 adet 1.5 altı değerden kaç tanesini doğru buldu
- **Hedef:** %75+
- **Neden önemli:** 1.5 altını bulamazsak bahis yapamayız → fırsat kaçırırız

### 🟢 1.5 ÜSTÜ DOĞRULUĞU

**Ne anlama geliyor?**
- Model, **gerçekten 1.5 üstünde olan** değerleri ne kadar doğru tahmin ediyor?
- Örnek: 100 adet 1.5 üstü değerden kaç tanesini doğru buldu
- **Hedef:** %75+
- **Neden önemli:** Doğru tahmin edersek gereksiz bahisten kaçınırız

### 💰 PARA KAYBI RİSKİ

**Ne anlama geliyor?**
- Model **1.5 altı olduğunda** yanlışlıkla "1.5 üstü" deme oranı
- Örnek: 100 oyunun kaçında yanlış tahminle **PARA KAYBEDERİZ!**
- **Hedef:** <%20
- **Neden önemli:** Bu metrik direk para kaybı riski!

### 🎯 Model Durumları

**✅ İYİ Durum (Hedef):**
```
🔴 1.5 ALTI: 75.0% ✅
🟢 1.5 ÜSTÜ: 80.0% ✅
💰 Para kaybı: 15.0% ✅
```
→ Model **dengelenmiş ve güvenli**

**⚠️ ORTA Durum (Gelişiyor):**
```
🔴 1.5 ALTI: 54.3% ⚠️
🟢 1.5 ÜSTÜ: 42.9% ⚠️
💰 Para kaybı: 45.7% ⚠️
```
→ Model **öğreniyor ama henüz hedefte değil**

**❌ KÖTÜ Durum (Kilitleniyor):**
```
🔴 1.5 ALTI: 0.0% ❌
🟢 1.5 ÜSTÜ: 100.0% ❌
💰 Para kaybı: 100.0% ❌
```
→ Model **sadece "1.5 üstü" tahmin ediyor** - Lazy learning!

---

## 🚀 Sonraki Adımlar

1. İndirilen dosyaları lokal projenize kopyalayın:
   - `jetx_progressive_final.h5` → `models/jetx_model.h5`
   - `scaler_progressive.pkl` → `models/scaler.pkl`

2. Streamlit uygulamasını yeniden başlatın:
   ```bash
   streamlit run app.py
   ```

3. Artık model 1.5 altı değerleri tahmin edebilecek!

---

## ⚠️ Önemli Notlar

- 🚨 Bu model %100 doğru değildir
- 💰 Yine de para kaybedebilirsiniz
- 🎯 1.5x kritik eşiktir
- 🛡️ Rolling mod en güvenlidir (%80+ güven)
- ⚡ Aggressive mod çok risklidir

**Sorumlu oynayın!** 🎲